# README

This conversion script is incomplete. Namely, symmetry information is not yet converted. Please see TODOs.

## Documentation of Format

The IPD native format and BOP format uses similar terminology in different ways. Here are some of the differences.

- IPD parts are considered BOP objects
    - See generated `ipd_part_to_bop_obj_id.json` for mapping
- IPD datasets are considered BOP scenes
    - See generated `ipd_dataset_to_bop_scene.json` for mapping
- IPD objects (referenced by part and instance) are considered BOP ground truth instances.
    - See generated `test[camera]/[bop_scene/ipd_obj_to_bop_gt_id.json` for mapping

Conversion details:
- IPD cameras are considered different BOP `split_type`s
- IPD lighting conditions are specified by different `test_targets`

```
/ipd
######## BASE ZIP
├─ camera_photoneo.json
├─ camera_basler_hr.json
├─ camera_basler_lr.json
├─ camera_flir_polar.json
├─ ipd_part_to_bop_obj_id.json
	- mapping from part name to BOP OBJ_ID
├─ ipd_dataset_to_bop_scene.json
	- mapping from dataset_id / background to BOP SCENE_ID
├─ test_targets_bop19_[all, room, day, spot].json
	- instances for each object in each scene, in each dataset, for different subsets of lighting conditions
########

######## MODELS ZIP
├─ models
│  ├─ models_info.json
│  ├─ obj_OBJ_ID.ply
├─ models_stl
│  ├─ models_info.json
│  ├─ obj_OBJ_ID.stl
########

######## PHOTONEO ZIP
├─ test_photoneo
│  ├─ BOP SCENE_ID
│  │  ├─ scene_camera.json
		- camera info for each IMG_ID
│  │  ├─ scene_gt.json
		- List[6D pose and OBJ_ID in GT_ID order] for each IMG_ID
│  │  ├─ scene_gt_info.json
		- List[bounding boxes in GT_ID order] for each IMG_ID
│  │  ├─ depth
│  │  │  ├─ IMGID.png
│  │  ├─ mask
│  │  │  ├─ IMGID_GTID.png
│  │  ├─ mask_visib
│  │  │  ├─ IMGID_GTID.png
│  │  ├─ rgb #if multiple resolutions, combined as HDR image
│  │  │  ├─ IMGID.png
######## 

######## [OTHER CAMERAS] ZIP
...
########
```

## To Install

### Option 1: Install Dev Deps via PDM
1. Install pdm 
2. Clone `ipd` repo 
3. Sync `bop_toolkit` submodule: `git submodule update --init --recursive`
4. `pdm install`

Note: `bop_toolkit` should be an editable install!

### Option 2: Manual Install via pip

#### IPD Toolkit

In [5]:
# !python3 -m pip install -e .

#### BOP Toolkit

In [ ]:
# !git clone git@github.com:thodan/bop_toolkit.git
# !python3 -m pip install -r bop_toolkit/requirements.txt -e bop_toolkit/

#### Other

In [ ]:
# !python3 -m pip install open3d, pymeshlab

# Begin Conversion

## 0. Imports & Setup

In [11]:
import os, shutil
import numpy as np
import open3d as o3d
import pymeshlab
import cv2
import json
from collections import defaultdict

In [12]:
from intrinsic_ipd import IPDReader
import intrinsic_ipd.constants

In [13]:
from bop_toolkit.bop_toolkit_lib import misc, inout

In [14]:
import logging
logging.basicConfig(
    level=logging.INFO,  # Set the logging level (INFO, DEBUG, WARNING, etc.)
    format="%(asctime)s - %(name)s - %(levelname)s - %(filename)s - %(funcName)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

In [15]:
def json_load_if_exists(path_to_json, default):
    if os.path.exists(path_to_json):
        with open(path_to_json, 'r') as fp:
            return json.load(fp, object_hook = lambda d: {int(k) 
                         if k.lstrip('-').isdigit() else k: v for k, v in d.items()})
    else:
        return default

### Read Dataset & Setup Destinations

In [16]:
camera = intrinsic_ipd.constants.IPDCamera.PHOTONEO #TODO: CHANGE ME!

#Choose dataset
bop_scene = 0 # TODO: CHANGE ME!
datasets = intrinsic_ipd.constants.DATASET_IDS
dataset = datasets[bop_scene]

In [17]:
# Read/Download dataset
reader = IPDReader("./datasets", dataset, camera, lighting=intrinsic_ipd.constants.IPDLightCondition.ALL, download=True) 

corner_bracket5.stl: 0.00B [00:00, ?B/s]
ERROR:root:url: https://storage.googleapis.com/akasha-public/industrial_plenoptic_dataset/cad_models/corner_bracket5.stl failed to download with error: HTTP Error 404: Not Found


In [18]:
# Make ipd dest
bop_dest = "./bop_datasets"
ipd_dest = os.path.join(bop_dest, "ipd")
os.makedirs(ipd_dest, exist_ok=True)

# Make camera dest
camera_dest = os.path.join(ipd_dest, f"test_{camera.name.lower()}")
os.makedirs(camera_dest, exist_ok=True)


In [19]:
# Make bop_scene map
bop_scene_map_file = os.path.join(ipd_dest, "ipd_dataset_to_bop_scene.json")
bop_scene_map = json_load_if_exists(bop_scene_map_file, {did : i for i, did in enumerate(intrinsic_ipd.constants.DATASET_IDS)})
inout.save_json(bop_scene_map_file, bop_scene_map)

In [20]:
# Make dataset (bop_scene) dest
scene_dest = os.path.join(camera_dest, f"{bop_scene:06}")
os.makedirs(scene_dest, exist_ok=True)

## 1. Convert Models

In [34]:
# Make models dest
models_stl_dest = os.path.join(ipd_dest, "models_stl")
models_dest = os.path.join(ipd_dest, "models")
# models_eval_dest = os.path.join(ipd_dest, "models_eval")
os.makedirs(models_stl_dest, exist_ok=True)
os.makedirs(models_dest, exist_ok=True)
# os.makedirs(models_eval_dest, exist_ok=True)

In [35]:
# CONVERT USING PYMESHLAB
def pymeshlab_stl_to_ply(in_stl_path, out_ply_path):
    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(in_stl_path)
    ms.save_current_mesh(out_ply_path,
                         binary = False,
                         save_vertex_normal = True
                         )

# Alternative: CONVERT USING OPEN3D
def o3d_stl_to_ply(in_stl_path, out_ply_path, sample=False, num_points = 10000):
    mesh = o3d.io.read_triangle_mesh(in_stl_path)
    if sample:
        cloud = mesh.sample_points_uniformly(num_points, use_triangle_normal=True)
    else:
        mesh.compute_vertex_normals()
        mesh.paint_uniform_color((1, 0.75, 0))
        mesh.compute_vertex_normals()
        cloud = o3d.geometry.PointCloud()
        cloud.points = mesh.vertices
        cloud.normals = mesh.vertex_normals 
    o3d.io.write_point_cloud(out_ply_path, cloud, write_ascii=True)



In [36]:
TEST = False

In [37]:
models_info = {}
sample = False
parts = intrinsic_ipd.constants.PART_NAMES
if TEST:
    parts = ['gear2']

part2obid = {}
for obj_id, part in enumerate(parts):
    obj_id += 1 # index starts with 1
    part2obid[part] = obj_id

    ######## SAVE MODEL
    # copy stl model to models_stl
    model_stl_path = os.path.join(reader.root, 'models', f'{part}.stl')
    if not os.path.exists(model_stl_path): 
        print(f"NO STL FILE FOUND FOR: {part} at {model_stl_path}")
        continue
    dst = os.path.join(models_stl_dest, f'obj_{obj_id:06}.ply')
    shutil.copyfile(model_stl_path, dst)

    # create ply model in models
    model_ply_path = os.path.join(models_dest, f'obj_{obj_id:06}.ply')
    pymeshlab_stl_to_ply(model_stl_path, model_ply_path)
    
    # TODO: create eval ply model in models_eval
    # - 'Uniformly' resamples and decimates 3D object models for evaluation. 
    # - See bop_toolkit/scripts/remesh_models_for_eval.py
    # !!!! DOES NOT WORK !!!! some error with pymeshlab. Try bop_toolkit/scripts/remesh_models_for_eval.py
    if False:
        ms = pymeshlab.MeshSet()
        ms.load_new_mesh(model_ply_path)
        ms.load_filter_script('./bop_toolkit/scripts/meshlab_scripts/remesh_for_eval_cell=0.25.mlx')
        ms.apply_filter_script()
        ms.save_current_mesh(os.path.join(models_eval_dest, f'obj_{obj_id:06}.ply'),
                            binary = False,
                            save_vertex_normal = True
                            )

    ######## SAVE MODEL INFO: see bop_toolkit/scripts/calc_model_info.py
    model = inout.load_ply(model_ply_path)
    ref_pt = list(map(float, model["pts"].min(axis=0).flatten()))
    size = list(map(float, (model["pts"].max(axis=0) - ref_pt).flatten()))
    diameter = misc.calc_pts_diameter(model["pts"])

    model_info = {
        "min_x": ref_pt[0],
        "min_y": ref_pt[1],
        "min_z": ref_pt[2],
        "size_x": size[0],
        "size_y": size[1],
        "size_z": size[2],
        "diameter": diameter,
    }

    # TODO: process symmetries
    # see: https://github.com/thodan/bop_toolkit/blob/97badc48dae87d03fa86c0f4ccce94ffdaaae4c5/bop_toolkit_lib/misc.py#L47
    
    symm = reader._get_symm_params(part)
    # print(symm)

    # # TODO: list of continuous symmetries arrays
    # model_info['symmetries_discrete'] = [
    #     np.eye(4).flatten() # 4x4 matrix flattened!
    # ]

    # # TODO: list of continuous symmetries dictionaries
    # model_info['symmetries_continuous'] = [
    #     {
    #         "axis": [ 0, 0, 1 ],
    #         "offset": [ 0, 0, 0 ]
    #     }
    # ]
    models_info[obj_id] = model_info

inout.save_json(os.path.join(models_dest, 'models_info.json'), models_info)
inout.save_json(os.path.join(models_stl_dest, 'models_info.json'), models_info)
# inout.save_json(os.path.join(models_eval_dest, 'models_info.json'), models_info) # TODO: create eval ply

inout.save_json(os.path.join(ipd_dest, 'ipd_part_to_bop_obj_id.json'), part2obid)

NO STL FILE FOUND FOR: corner_bracket5 at ./datasets/models/corner_bracket5.stl


## 2. Convert IPD object into BOP ground truth instances

In [28]:
def get_bop_gt_id_map(reader, scene_dest):
    map_json = os.path.join(scene_dest, 'ipd_obj_to_bop_gt_id.json')

    bop_gt_id_map = json_load_if_exists(map_json, defaultdict(dict))

    for i, obj in enumerate(reader.objects):
          bop_gt_id_map[obj[0]][obj[1]] = i
    with open(map_json, 'w') as fp:
            json.dump(bop_gt_id_map, fp, sort_keys=True, indent=4)
    return bop_gt_id_map

get_bop_gt_id_map(reader, scene_dest)

{'gear1': {0: 0, 1: 1, 2: 2, 3: 3},
 'pegboard_basket': {0: 4},
 'u_bolt': {0: 5, 1: 6, 2: 7}}

## 3. Move/Convert Images

In [26]:
# Make image dests
depth_dest = os.path.join(scene_dest, 'depth')
mask_dest = os.path.join(scene_dest, 'mask_ipd')
rgb_dest = os.path.join(scene_dest, 'rgb')

os.makedirs(depth_dest, exist_ok=True)
os.makedirs(mask_dest, exist_ok=True)
os.makedirs(rgb_dest, exist_ok=True)

### Move RBGD images

In [44]:
def merge_exposures(img_paths):
    img_list = [cv2.imread(path) for path in img_paths]
    exposure_times = np.array([1, 30, 80, 200], dtype=np.float32)
    merge_debevec = cv2.createMergeDebevec()
    hdr_debevec = merge_debevec.process(img_list, times=exposure_times.copy())
    return hdr_debevec

For photoneo, will move & rename rgb and depth files.

For other cameras, will merge into an hdr photo and save

In [45]:
for bop_image_id in reader.scenes.keys():
    # move or merge rgb & depth photos
    if reader.camera == intrinsic_ipd.IPDCamera.PHOTONEO:
        from_path = reader._get_img_file(bop_image_id, intrinsic_ipd.IPDImage.PHOTONEO_DEPTH)
        to_path = os.path.join(depth_dest, f'{bop_image_id:06}.png')
        shutil.copy(from_path, to_path)

        from_path = reader._get_img_file(bop_image_id, intrinsic_ipd.IPDImage.PHOTONEO_HDR)
        to_path = os.path.join(rgb_dest, f'{bop_image_id:06}.png')
        shutil.copy(from_path, to_path)
    else:
        img_paths = [reader._get_img_file(bop_image_id, image_type) for image_type in reader.camera.images]
        hdr_photo = merge_exposures(img_paths)
        to_path = os.path.join(rgb_dest, f'{bop_image_id:06}.png')
        cv2.imwrite(to_path, hdr_photo)
    


### Move masks based on ground truth id

Move and rename masks based on ground truth id.

In [29]:
# move and rename mask photos
bop_gt_id_map = None
for bop_image_id in reader.scenes.keys():
    for object in reader.objects:
        part, instance = object
        bop_gt_id_map = get_bop_gt_id_map(reader, scene_dest)
        bop_gt_id = bop_gt_id_map[part][instance]
        _, ipd_mask_path = reader.get_mask(bop_image_id, part, instance, return_path=True)
        bop_mask_path = os.path.join(mask_dest, f'{bop_image_id:06}_{bop_gt_id:06}.png')
        shutil.copy(ipd_mask_path, bop_mask_path)

## 4. Process Labels

### scene_camera.json
cam_K, cam_R_w2c, cam_t_w2c, depth_scale, elev, mode 
for each bop image (ipd scene)

TODO: Read and add to this file when loading other lighting conditions??? (or will that be a new type?)

In [63]:
scene_camera_path = os.path.join(scene_dest, 'scene_camera.json')
camera_path = os.path.join(ipd_dest, f"camera_{reader.camera.name.lower()}.json")

In [64]:
w2c = np.linalg.inv(reader.cam_c2w)
camera_info = {
    'cam_K': reader.cam_K.flatten().tolist(),
    'cam_R_w2c': w2c[:3,:3].flatten().tolist(),
    'cam_t_w2c': w2c[:3,3].flatten().tolist(),
}

if reader.camera is intrinsic_ipd.constants.IPDCamera.PHOTONEO:
    camera_info['depth_scale']= 1.0

image = reader.get_img(list(reader.scenes.keys())[0])
height, width = image.shape[:2]

scene_camera = {bop_image_id : camera_info  for bop_image_id in reader.scenes.keys()}
inout.save_json(scene_camera_path, scene_camera)
inout.save_json(camera_path, {
  "cx": reader.cam_K[0, 2],
  "cy": reader.cam_K[1, 2],
  "depth_scale": 1.0,
  "fx": reader.cam_K[0, 0],
  "fy": reader.cam_K[1, 1],
  "height": height,
  "width": width
})

### scene_gt.json
map IMG_ID to List[6D pose and OBJ_ID in GT_ID order]

In [49]:
scene_gt_path = os.path.join(scene_dest, 'scene_gt.json')

In [50]:
def get_gt_info(reader, ipd_scene_id, ipd_dest, scene_dest):
    with open(os.path.join(ipd_dest, "ipd_part_to_bop_obj_id.json"), 'r') as fp:
        bop_obj_id_map = json.load(fp)
    ipd_objects = reader.objects
    o2c = reader.o2c.sel(scene=ipd_scene_id)
    gt_info = {}
    bop_gt_id_map = get_bop_gt_id_map(reader, scene_dest)
    for part, instance in ipd_objects:
        gt_id = bop_gt_id_map[part][instance]
        gt_o2c = o2c.sel(part=part, instance=instance).data
        obj_id = bop_obj_id_map[part]
        gt_info[gt_id] = {
            'obj_id': obj_id, 
            'cam_R_m2c': gt_o2c[:3,:3].flatten().tolist(),
            'cam_t_m2c': gt_o2c[:3, 3].flatten().tolist(),
            'gt_id': gt_id,
            'ipd_object': (part, instance)
        }
    gt_keys = [int(k) for k in gt_info.keys()]
    max_gt_id = max(gt_keys)
    return [gt_info.get(gt_id, {}) for gt_id in range(max_gt_id)]


In [51]:
scene_gt = {bop_image_id: get_gt_info(reader, bop_image_id, ipd_dest, scene_dest) for bop_image_id in reader.scenes.keys()}
inout.save_json(scene_gt_path, scene_gt)

#### test_targets.json

In [62]:
import itertools, operator

bop_obj_id_map = json_load_if_exists(os.path.join(ipd_dest, 'ipd_part_to_bop_obj_id.json'), {})


conditions = [intrinsic_ipd.constants.IPDLightCondition.ALL,
              intrinsic_ipd.constants.IPDLightCondition.DAY, 
              intrinsic_ipd.constants.IPDLightCondition.ROOM,
              intrinsic_ipd.constants.IPDLightCondition.SPOT]

bop_scene = json_load_if_exists(os.path.join(ipd_dest, 'ipd_dataset_to_bop_scene.json'), {})[reader.dataset_id]
for condition in conditions:
    target_file = os.path.join(ipd_dest, f'test_targets_bop19_{condition.name.lower()}.json')
    targets = json_load_if_exists(target_file, [])    
    for part, objects in itertools.groupby(reader.objects, operator.itemgetter(0)):
        obj_id = bop_obj_id_map[part]
        inst_count = len(list(objects))
        for bop_image_id in condition.scenes:
            target = {
                "im_id": bop_image_id,
                "obj_id": obj_id,
                "inst_count": inst_count,
                "scene_id": bop_scene,
            }
            if target not in targets:
                targets.append(target)
    
    inout.save_json(target_file, targets)

    targets_24_file = os.path.join(ipd_dest, f'test_targets_bop24_{condition.name.lower()}.json')
    targets_24 = json_load_if_exists(targets_24_file, [])
    for bop_image_id in condition.scenes:
        target_24 = {
                "im_id": bop_image_id,
                "scene_id": bop_scene,
            }
        if target_24 not in targets_24:
            targets_24.append(target_24)
    inout.save_json(targets_24_file, targets_24)
    

## 5. Repeat above steps for all dataset ids and camera

## 6.Run BOP scripts to generate rest of dataset info 

To run the bop_toolkit scripts, need to make some edits:

Changelog to `bop_toolkit` as reflected in @carynbear's fork:
- in `bop_toolkit/bop_toolkit_lib/dataset_params.py`
    - CHANGED: added `ipd` params throughout
    - TODO: indicate which objects (parts) have symmetry.
    - TODO: get sizes of images for other cameras (photoneo done)
    - TODO: calculate depth_range, azimuth_range, elev_range 
- in `bop_toolkit/bop_toolkit_lib/config.py`
    - CHANGED: `output_path`
- in `bop_toolkit/scripts/calc_gt_info.py`
    - CHANGED: run with `ipd` and `vis`
    - CHANGED: try catch to skip missing cad models
- in `bop_toolkit/scripts/calc_gt_masks.py`
    - CHANGED: run with `ipd`
    - CHANGED: try catch to skip missing cad models

In [23]:
os.environ["BOP_PATH"] = bop_dest

### Calculate scene_gt_info.json
map IMG_ID to List[bounding boxes in GT_ID order]

In [22]:
!python3 ./bop_toolkit/scripts/calc_gt_info.py

9/20|22:18:08: Initializing renderer...
vispy uses app: egl, gl: gl2
9/20|22:18:10: Calculating GT info - dataset: ipd (test, None), scene: 0, im: 0


### Generate masks from gt labels

In [25]:
!python3 ./bop_toolkit/scripts/calc_gt_masks.py

9/20|22:27:31: Initializing renderer...
vispy uses app: egl, gl: gl2
9/20|22:27:32: Calculating masks - dataset: ipd (test, None), scene: 0, im: 0


### Generate models_evel with pymeshlab
- TODO: Specify the meshlab_server_path to the executable in `bop_toolkit/bop_toolkit_lib/config.py`
- TODO: Edit the script to run with `ipd` dataset!

In [30]:
# !python3 ./bop_toolkit/scripts/remesh_models_for_eval